# Django Object Relational Mapper (ORM)

## Introduction

This notebook provides a brief overview of the [Django](https://docs.djangoproject.com/en/3.1/) [ORM](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping), which allows us to access the database using Python objects. We will assume you have some working knowledge of Python, databases ([PostgreSQL](https://www.postgresql.org/)) and [object-oriented programming](https://pythonguides.com/object-oriented-programming-python/). New concepts relating to the Django ORM will be introduced as we work through the notebook.

The objective of the notebook is to get you familiar with Python commands that you can use for accessing the data. By the end of the notebook hopefully you will understand: 

* How to construct queries to the data
* How to write data to the database
* How to write post-processing scripts using the data

using Python with Django. There are cells in the notebook that are intentionally empty. These are left blank so you can fill them out to test your understanding of Django syntax and practice writing the code.

## Initial Setup

To get started, run the cell below which will configure our Python environment to have access to Django shell functionality. **You must run this cell before any other cells**.

In [ ]:
# Setup environment
import sys
import os
import django
sys.path.append('src')
os.environ['DJANGO_SETTINGS_MODULE'] = 'demo.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Context

For the purposes of this tutorial let's suppose we are managers of book shop franchise. There are a number of stores and books that we stock. We've got a database system to reflect the physical system (because that's what everyone else is doing). 

## Models

The most important concept in the ORM is the [Django Model](https://docs.djangoproject.com/en/3.1/topics/db/models/). Models are a pythonic representation of a database table. They contain information about the name and data type for each field in a table. For those interested, here is how the `models.py` files for our books, stores, inventory and sales look:

```
# src/stores/models.py
class Store(models.Model):
    id = models.AutoField(primary_key=True)
    name = models.CharField(max_length=100)
    address = models.TextField()

# src/books/models.py
class Book(models.Model):
    id = models.AutoField(primary_key=True)
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=200)
    price = models.FloatField()
    store = models.ForeignKey(Store, to_field='id', null=True, blank=True, on_delete=models.SET_NULL)
    
# src/sales/models.py
class Sale(models.Model):
    id = models.AutoField(primary_key=True)
    book = models.ForeignKey(Book, to_field='id', on_delete=models.DO_NOTHING)
    store = models.ForeignKey(Store, to_field='id', on_delete=models.DO_NOTHING)
    sold_at = models.DateTimeField(auto_now=True)
    
# src/inventory/models.py
class Inventory(models.Model):
    id = models.AutoField(primary_key=True)
    book = models.ForeignKey(Book, to_field='id', on_delete=models.DO_NOTHING)
    store = models.ForeignKey(Store, to_field='id', on_delete=models.DO_NOTHING)
    quantity = models.IntegerField()
```

This reflects the types of data contained in each table in the database. From the model definitions, you can immediately underestand the database schema. These files can also be generated automatically from a database!

This [page](https://docs.djangoproject.com/en/3.1/topics/db/models/) provides an in-depth reference for Django Models that you can read for more information.

# Simple Queries

### Check our inventory!

First step will be to check what books we currently have stocked across all of our book stores. Actually, we don't even know what book stores we have, so let's check that too. In order to access the data, you will need to import the relevant Django model. Run the cell below to import the models and run a simple query. 

In [ ]:
from books.models import Book
from stores.models import Store

In [ ]:
all_books = Book.objects.all()
all_stores = Store.objects.all()

print(all_books)
print(all_stores)

We can see that the model objects have a `.objects.all()` operation that you can apply to them. This returns the objects in a `QuerySet` which is effectively just a list. In this case, since we queried for `all()`, the result is all rows of the table. If we perform some filtering on the query we will get a subset of the rows in our `QuerySet`.

Lets look at one of the books. We can retrieve items (rows) from our `QuerySet` by indexing, like we do for lists. 

In [ ]:
print(all_books[0])
print(type(all_books[0]))

The book, which is a row in the database, is represented as the Django model object defined above. When it is printed, it just shows `books.models.Book` which is not very informative. Let's look at the properties of this object.

In [ ]:
all_books.first().__dict__

We can retrieve the information about the book by inspecting this `__dict__` attribute. This is handy for looking at all properties of an object (looking at the values of all fields in the row).

If we wanted to write some script that makes use of a specific property, there is an easier way to access them. Let's print the names of all the books we have in our store.

In [ ]:
for book in Book.objects.all():
    print(book.title)

We also have a Store table in the database with one entry. What is the name of the store?

In [ ]:
# TODO(user): Retrieve the name of the Store object in the database.




Cool! So we know how to get all rows from the database for a table of interest. We know how to access objects in that list, and look at properties of those objects. 

# Write data

### Opening a new shop

Let's suppose we've just opened up a new shop. We need to update the database to reflect this change.

In [ ]:
Store.objects.create(
    name="Awesome Books",
    address="36 Stirling Hwy, Perth, AU 6009"
)

Now if we look at all of the stores, you will see that we have two. Do this in the cell below.

In [ ]:
# TODO(user): Check that there are now two stores. Check the names of each of the stores
# to make sure one of them is the one you just created




**NOTE**: If you run a cell twice you will accidentally create two objects! This will result in two writes to the database with the same content. If you accidentally do this, no worries. There are no protections in this database that require no duplication of fields, but there probably will be in a production database.

To access the object where you have created duplicated you can use the `id` field of the object. This is the row id, and is unique for each object. In Django, this is:

``` 
crawley_store = Store.objects.get(id=2)
```

Don't you think the name of our first book shop is super boring? Let's change it to something more memorable. We can do that by updating that row in the database. Let's do that through Django.

In [ ]:
perth_store = Store.objects.get(id=1)
perth_store.name = "Exciting Book Shop"
perth_store.save()

Updating the data is as simple as changing the value of the object property. 

**NOTE**: Once you have updated the object you will need to save it with the `.save()` method. Otherwise, the changes will not be recorded in the database.

### Add stock to the existing shop!

So we have now two shops and some number of books. Since our database is [normalised](https://en.wikipedia.org/wiki/Database_normalization), the table that tracks the number of books we have in stock is separate to the Books table. We haven't had the chance yet to look at our inventory, so let's do that quickly.

In [ ]:
from inventory.models import Inventory

for item in Inventory.objects.all():
    print(f"({item.quantity}, {Book.objects.get(id=item.book_id).title}, {Store.objects.get(id=item.store_id).name})\n")

Great so now we know how much of each book we currently have in each store. Let's add some stock to the new bookshop.

In [ ]:
# Adding some Books to the new Store by updating the Inventory table.

Inventory.objects.create(
    quantity=10,
    book_id=1,
    store_id=2
)

Inventory.objects.create(
    quantity=15,
    book_id=4,
    store_id=2
)

Inventory.objects.create(
    quantity=15,
    book_id=6,
    store_id=2
)

### New books!

We've got a new book that we want to add to our shop in Crawley. There are a lot of students in Crawley so we decided to add a book that targets them. Now that you've seen how to create items in the Inventory, you will fill out the content for this cell. The information for the book is here:

```
title = "This Side of Paradise"
author = "Francis Scott Fitzgerald"
price = 22.99
```

In [ ]:
# TODO(user): Create Book and add the Inventory row





Now if you have added the Book and Inventory correctly you should see 4 different items in the new Crawley book store.

In [ ]:
# TODO(user): Run this to check that you have added the book and inventory rows to the database
# NOTE: If you ran the cell above multiple times this should still show True.

print(f"Correct number of books in the Crawley store? {Inventory.objects.filter(store_id=2).count() >= 4}")
print(f"Correct number of books in the all stores? {Inventory.objects.count() >= 10}")

# Complex Queries

### Generating sales

Lets generate some random sales to showcase the ability to perform some complex queries. We're going to generate a lot of sales to populate the database over the last 10 years.

In [ ]:
import random
import pytz
from datetime import datetime, timedelta
from sales.models import Sale

In [ ]:
# Create some sales records in the database

# variables for generating random datetime values
min_year = 2010
max_year = datetime.now(tz=pytz.UTC).year
start = datetime(min_year, 1, 1, 0, 0, 0)
years = max_year - min_year + 1
end = start + timedelta(days=365 * years)

# Generate sales
N = 2000
for i in range(N):
    random_date = start + (end - start) * random.random()
    sale = Sale.objects.create(
        book_id = random.randint(1, Book.objects.count()),
        store_id = random.randint(1, Store.objects.count())
    )
    sale.sold_at = random_date
    sale.save()

In [ ]:
# Confirm that we have a lot of sales

Sale.objects.count()

Now let's try and figure out how much money we made from the book 'Sapiens' in the last year. We're not sure if we should keep that one in stock since we hear that people are not reading that sort of stuff anymore. Once we've done that, we'll compare that with the amount made by other books. The cell below calculates the total amount made from the book Sapiens.

In [ ]:
# Calculate total sales from book "Sapiens" with id=1

sapiens_sale_count = Sale.objects.filter(book_id=1).filter(sold_at__gte=datetime.today() - timedelta(days=365*1)).count()
print(f"Total sales for book Sapiens: ${round(sapiens_sale_count * Book.objects.get(id=1).price, 2)}")

We can see that the basic idea for creating complex filters is to use `.filter()` and write your logic (which can be chained to specify multiple conditions). There is comprehensive documentation on `filter` which you can find [here](https://docs.djangoproject.com/en/3.1/topics/db/queries/#retrieving-specific-objects-with-filters). The section of the documentation specifying the filtering commands (`gte`, `lte` etc) is found [here](https://docs.djangoproject.com/en/3.1/topics/db/queries/#field-lookups).

We could even see how much revenue each book store generated for their sales of the book.

In [ ]:
# How much each book store earned in the sale of Sapiens in the last year

sapiens_book_id = 1 
for store in Store.objects.all():
    copies_sold = Sale.objects.filter(store_id=store.id)\
                    .filter(book_id=sapiens_book_id)\
                    .filter(sold_at__gte=datetime.today() - timedelta(days=365*1))\
                    .count()
    revenue = copies_sold * Book.objects.get(id=sapiens_book_id).price
    print(f"Store '{store.name}' made ${round(revenue, 2)} from {Book.objects.get(id=sapiens_book_id).title} sales.")

# Analysis

Suppose we want to understand our sales better and communicate that to a bunch of potential investors (so that we can expand and essentially overtake Amazon). We can use the data in our database to produce the figures. We'll use the [matplotlib](https://matplotlib.org/stable/index.html) plotting library to make the figures.

Let's start by producing a bar chart of the revenue of each book across the stores. We'll use the authors names for the axes labels since the book names are too long.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Retrieve data in the format necessary.

revenue = [book.price * Sale.objects.filter(book_id=book.id).count() for book in Book.objects.all()]
labels = [book.author for book in Book.objects.all()]

plt.xticks(range(len(revenue)), labels, rotation='vertical')
plt.xlabel('Book (Author)')
plt.ylabel('Revenue [$]')
plt.title('Revenue by Book')
plt.bar(range(len(revenue)), revenue) 
plt.show()

Great! Now suppose we want a plot to show how the sales have varied through time for each of the books. Let's look at the sales for each book each year. We'll use a [pandas](https://pandas.pydata.org/) dataframe to structure our data.

In [ ]:
import pandas as pd

In [ ]:
# Create data frame of sales
revenue_df = pd.DataFrame()
years = list(range(2010, datetime.now(tz=pytz.UTC).year))

for book in Book.objects.all():
    sales = [round(book.price * Sale.objects.filter(book_id=book.id).filter(sold_at__year=year).count(), 2) for year in years]
    revenue_df[book.title] = sales

In [ ]:
# Plotting data frame

years = list(range(2010, datetime.now(tz=pytz.UTC).year))

for (col_name, col_data) in revenue_df.iteritems(): 
    plt.plot(years, col_data, label=col_name)
    plt.scatter(years, col_data)
    
plt.rcParams['figure.figsize'] = [20, 10]
plt.title('Revenue across the years by book')
plt.xlabel('Book (Author)')
plt.ylabel('Revenue [$]')
plt.legend()
plt.show()

Cool, so we can see the sales appear to be sort of random which is what we expect since we generated them with a random generator lol.

# Wrapping up

Great! So in this tutorial we've learned how to:

* Perform database queries through the Django ORM
* Write to the database
* Update data in the database
* Structure complex queries for analysis

Hopefully it's been valuable for you to go through this. Feel free to contact me if you have any feedback.